## Deezer playlist dataset et recomandandation de morceaux avec word2vec

Dans ce mini projet nous allons mettre au point un réseau word2vec et nous en servir pour construire un outils de complétion de playlist (suggestion de morceaux). Les données sont hébergée sur le dépot suivant : http://github.com/comeetie/deezerplay.git. Pour en savoir plus sur word2vec et les données que nous allons utiliser vous pouvez lire les deux références sivantes :

- Efficient estimation of word representations in vector space, Tomas Mikolov, Kai Chen, Greg Corrado, and Jeffrey Dean. (https://arxiv.org/abs/1301.3781)
- Word2vec applied to Recommendation: Hyperparameters Matter, H. Caselles-Dupré, F. Lesaint and J. Royo-Letelier. (https://arxiv.org/pdf/1804.04212.pdf)

Les éléments que vous devez réaliser sont mis en évidence en <span style="color:red">rouge</span> 


### Préparation des données
Les données sont sous la forme d'une liste de playlist. Chaque playlist est elle me une liste avec l'identifiant deezer du morçeau suivi de l'identifiant de l'artiste.

In [83]:
# chargement des données de playlist
import numpy as np
data = np.load("./music_2.npy",allow_pickle=True)
[len(data), np.mean([len(p) for p in data])]

[100000, 24.21338]

Le jeu de données sur lequel nous allons trvailler contient 100000 playlist qui sont composeer d'en moyenne 24.1 morceaux. Nous allons commencer par ne conserver que les identifiants de morceau. 

In [84]:
# separation des ids de morçeau et d'artist
playlist_track = [list(filter(lambda w: w.split("_")[0]==u"track",playlist)) for playlist in data]
playlist_artist = [list(filter(lambda w: w.split("_")[0]==u"artist",playlist)) for playlist in data]

In [85]:
# nombre de morceaux != ?
tracks = np.unique(np.concatenate(playlist_track))
Vt = len(tracks)
Vt

338509

In [86]:
playlist_track[1]

['track_354917971',
 'track_354917981',
 'track_354917991',
 'track_354918001',
 'track_354918011',
 'track_405992352',
 'track_383552261',
 'track_397746652',
 'track_385499681',
 'track_402932992',
 'track_398786652',
 'track_357037431',
 'track_378113751',
 'track_354918011',
 'track_403520542',
 'track_393157072',
 'track_354165291',
 'track_357083271',
 'track_361171651',
 'track_145501426',
 'track_397141342',
 'track_145370202']

Le nombre de morceaux différents dans ce data-set est assez élevé avec plus de 300 000 morceaux.

### Création d'un dictionnnaire de morceau
Nous allons affecter a chaque morceau un entier qui nous servira d'identifiant unique et d'entrée pour notre réseau. Pour économiser un peu nos ressources nous allons travailler dans ce TP que sur les morceaux qui apparaissent dans au moins deux playlists.

In [87]:
# nombre d'occurence de chaque morceaux ?
track_counts = dict((tracks[i],0) for i in range(0, Vt))
for p in playlist_track:
    for a in p:
        track_counts[a]=track_counts[a]+1;

In [88]:
# Filtrage des morceaux peu fréquent pour gangner un peu de temps au vue de nos ressource en temps de calcul  
playlist_track_filter = [list(filter(lambda a : track_counts[a]> 1, playlist)) for playlist in playlist_track]
# recupération des comptage
counts  =  np.array(list(track_counts.values()))
# trie
order = np.argsort(-counts)
# création de notre liste d'identifiant deezer
tracks_list_ordered = np.array(list(track_counts.keys()))[order]
# Taille de notre vocabulaire = nombre de morçeau conservés
Vt=np.where(counts[order]==1)[0][0]
# construction d'un dict id_morceaux id [0,Vt]
track_dict = dict((tracks_list_ordered[i],i) for i in range(0, Vt))
# conversion des playlist en liste d'entier
corpus_num_track = [[track_dict[track] for track in play ] for play in playlist_track_filter]

In [89]:
corpus_num_track[1]

[313,
 436,
 259,
 521,
 66,
 235,
 152,
 56,
 116,
 71,
 505,
 113,
 217,
 66,
 277,
 527,
 348,
 848,
 378,
 465,
 532,
 2016]

### Création des ensembles d'apprentissage de test et de validation

Pour apprendre les paramètre de notre méthode nous allons conserver les $l-1$ premiers morceaux de chaque playlist (avec $l$ la longueur de la playlist) pour l'apprentissage. Pour évaluer les performances de completion de notre méthode nous conservons pour chaque playlist les deux derniers morceaux. L'objectif sera de trouver le dernier a partir de l'avant dernier. 



In [90]:
# ensemble de test et d'apprentissage
index_tst = np.random.choice(100000,20000)
index_val = np.setdiff1d(range(100000),index_tst)
# le debut de chaque playlist est conservé pour l'apprentissage
play_app  = [corpus_num_track[i][:(len(corpus_num_track[i])-1)] 
             for i in range(len(corpus_num_track)) if len(corpus_num_track[i])>1]
# les deux derniers élemnts pour le test et la validation
play_tst  = np.array([corpus_num_track[i][(len(corpus_num_track[i])-2):len(corpus_num_track[i])] 
             for i in index_tst if len(corpus_num_track[i])>3])
play_val  = np.array([corpus_num_track[i][(len(corpus_num_track[i])-2):len(corpus_num_track[i])] 
             for i in index_val if len(corpus_num_track[i])>3])[:10000]


In [91]:
# import de Keras
from keras.models import Sequential, Model
from keras.layers import Embedding, Reshape, Activation, Input, Dense,Flatten
from keras.layers.merge import Dot
from keras.utils import np_utils
from keras.preprocessing.sequence import skipgrams

### hyper-paramètres de word2vec :

La méthode word2vec fait intervennir un certains nombre d'hyper paramètres. Nous allons les définirs et leurs donner des première valeurs que nous affinerons par la suite:

In [92]:
# dimension de l'espace latent
vector_dim = 30
# taille de la fenêtre de voisinage
window_width = 3
# sur-échantillonage des exemples négatifs
neg_sample = 5
# taille des mini-batch
min_batch_size = 50
# coeff pour la loi de tirage des exemple negatif
samp_coef = 0.5
# coeff pour le subsampling
sub_samp = 0.00001

# loss: 0.4250 - accuracy: 0.8508

### Création des tables de probabilité de tirage (lissée) et non lissée

Pour tirer les exmples négatif nous avons besoin des fréquence lissé de chaque morceau dans notre dataset. De même pour sous échantilloner les morceaux très fréquents nous avons besoin des fréquence brutes. Nous allons calculer ces deux vecteurs.

In [93]:
# recupération des comptage
counts = np.array(list(track_counts.values()),dtype='float')[order[:Vt]]
# normalisation
st =  counts/np.sum(counts)
# lissage
st_smooth = np.power(st,samp_coef)
st_smooth = st_smooth/np.sum(st_smooth)

### Construction du réseau word2vec

Un réseau word2vec prend en entrée deux entiers correspondant à deux morceaux, ceux-ci sont plonger dans un espace latent de dimension (vector_dim) grâce a une couche de type embedding (vous devrez utilisez la même couche pour projeter les deux morceaux). Une fois ces deux vecteurs extraits le réseau doit calculer leur produit scalaire normaliser appleler cosine distance : 

$$cos(\theta_{ij})=\frac{z_i.z_j}{||z_i||||z_j||}$$ 

Pour réaliser ce traitement vous utiliserez une couche "Dot" pour "dot product". Le modèle utilise ensuite une couche de type sigmoid pour produire la sortie. Cette sortie vaudra 0 lorsque les deux morceaux sont des morceaux tirés aléatoirement dans l'ensemble du jeu de donnée et 1 lorsqu'il aurront était extraits de la même playslist. <span style="color:red">A vous de créer le modèle keras Track2Vec correspondant à cette architecture.</span>

In [94]:
# entrée deux entier (couple de morceaux)
input_target = Input((1,), dtype='int32')
input_context = Input((1,), dtype='int32')

# a vous de compléter
embedded = Embedding(Vt, vector_dim, name="embedding-input")
embedded_target = embedded(input_target)
embedded_context = embedded(input_context)
dot_product = Dot(axes=1, normalize=True, name="dot")([embedded_target, embedded_context])
output = Dense(1, activation='sigmoid',name="classif")(dot_product)

# definition du modèle
Track2Vec = Model(inputs=[input_target, input_context], outputs=output)
Track2Vec.compile(loss='binary_crossentropy', optimizer='adam',metrics=["accuracy"])

In [95]:
Track2Vec.summary()

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_21 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 input_22 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 embedding-input (Embedding)    (None, 1, 30)        3697230     ['input_21[0][0]',               
                                                                  'input_22[0][0]']               
                                                                                                  
 dot (Dot)                      (None, 30, 30)       0           ['embedding-input[0][0]', 

### Création du générateur de données

Pour apprendre la couche de projection au coeur de notre modèle nous allons construire une générateur d'exemples positifs et négatifs de pair de morceaux proche ou aléatoires issues de nos données d'entrainement. La fonction suivante va permettre de générer de tels exemples a partir d'une playlist (seq) fournies en entrées. Cette fonction va tout d'abord construire tout les couples de morceau pouvant être extraient de la séquences s'ils se situent à moins de (windows) disance l'un de l'autres. Ces paires constitueront les paires positives. Les paires concernant deux morceaux très fréquents seront supprimer avec une probabilité qui dépendra de leur fréquences. Enfin un nombre d'exemple négatifs (correpondant neg_samples * nombre d'exemple positif) vont être tirés aléatoirement en utilisant la table de tirage (neg_sampling_table). 

In [96]:
# fonction générant les données associé a une séquence
# seq : séquence d'entrée
# neg_samples : nombre d'exemple négatif générés par example positif
# neg_sampling_table : probabilité de tirage des exemples négatif
# sub sampling_table : probabilité servant a sous échantilloner
# sub_t : paramètre de sous échantillonage
def word2vecSampling(seq,window,neg_samples,neg_sampling_table,sub_sampling_table,sub_t):
    # taille du vocabulaire
    V = len(neg_sampling_table)
    # créations des paires positives a partir de la séquence
    positives = skipgrams(sequence=seq, vocabulary_size=V, window_size=window,negative_samples=0)
    ppairs    = np.array(positives[0])
    # sous échantillonage
    if (ppairs.shape[0]>0):
        f = sub_sampling_table[ppairs[:,0]]
        subprob = ((f-sub_t)/f)-np.sqrt(sub_t/f)
        tokeep = (subprob<np.random.uniform(size=subprob.shape[0])) | (subprob<0)
        ppairs = ppairs[tokeep,:]
    nbneg     = ppairs.shape[0]*neg_samples
    # tirage des paires négatives
    if (nbneg > 0):
        negex     = np.random.choice(V, nbneg, p=neg_sampling_table)
        negexcontext = np.repeat(ppairs[:,0],neg_samples)
        npairs    = np.transpose(np.stack([negexcontext,negex]))
        pairs     = np.concatenate([ppairs,npairs],axis=0)
        labels    = np.concatenate([np.repeat(1,ppairs.shape[0]),np.repeat(0,nbneg)])
        perm      = np.random.permutation(len(labels))
        res = [pairs[perm,:],labels[perm]]
    else:
        res=[[],[]]
    return res

<span style="color:red">Utilisez cette fonction pour constuire un générateur "track_ns_generator" de données qui va générer des exemples positifs et négatifs à partir de "nbm" playlists tirées aléatoirement dans le jeu de données "corpus_num" fournis en entrée. </span>

In [97]:
# définition du générateur de couple de morceaux (y=0 <-> aléatoire, y=1 <-> proche dans une playlist)
import random
def track_ns_generator(corpus_num,nbm):
    #while True :
    # tirage de nbm playlist dans corpus_num
    x1 = []
    x2 = []
    y = []
    for i in range(nbm) :
        playlist = random.choice(corpus_num)
        # création des données x et y 
        xa, ya = word2vecSampling(playlist, window_width, neg_sample, st, st_smooth, sub_samp)
        for elem in xa :
            x1.append(elem[0])
            x2.append(elem[1])
        for elem in ya :
            y.append(elem)
    x = [np.array(x1), np.array(x2)]
    y = np.asarray(y).astype('float32').reshape((-1,1))
    print(len(y))
    print(len(x1))
    return (x,y) 

track_ns_generator(play_app, min_batch_size)

12252
12252


([array([18265,   418,   352, ..., 61308, 61308,  7944]),
  array([  5192,     93,    697, ...,    128,  28224, 102334])],
 array([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [1.]], dtype=float32))

## Apprentissage 
Vous devriez maintenant être en mesure d'apprendre votre premier modèle avec le code suivant. Cela devrait durer entre 15 et 30 min.

In [98]:
import tensorflow
xx, yy = track_ns_generator(play_app,min_batch_size)
tensorflow.config.run_functions_eagerly(True)
hist = Track2Vec.fit(xx, yy, steps_per_epoch = 200,epochs=60)

11394
11394
Epoch 1/60
200/200 [==============================] - 11s 55ms/step - loss: 0.8158 - accuracy: 0.5267
Epoch 2/60
200/200 [==============================] - 11s 53ms/step - loss: 0.6512 - accuracy: 0.6279
Epoch 3/60
200/200 [==============================] - 11s 53ms/step - loss: 0.5755 - accuracy: 0.7972
Epoch 4/60
200/200 [==============================] - 11s 53ms/step - loss: 0.5376 - accuracy: 0.8333
Epoch 5/60
200/200 [==============================] - 11s 54ms/step - loss: 0.5127 - accuracy: 0.8333
Epoch 6/60
200/200 [==============================] - 11s 53ms/step - loss: 0.4945 - accuracy: 0.8333
Epoch 7/60
200/200 [==============================] - 11s 54ms/step - loss: 0.4810 - accuracy: 0.8333
Epoch 8/60
200/200 [==============================] - 11s 53ms/step - loss: 0.4708 - accuracy: 0.8333
Epoch 9/60
200/200 [==============================] - 11s 53ms/step - loss: 0.4636 - accuracy: 0.8333
Epoch 10/60
200/200 [==============================] - 11s 54ms/step -

## Sauvegarde de l'espace latent
Nous pouvons une fois l'apprentissage effectué sauvegarder la position des morceaux dans l'espace latent avec le code suivant:

In [99]:
# récupérations des positions des morceaux dans l'espace de projection
vectors_tracks = Track2Vec.get_weights()[0]
with open('latent_positions.npy', 'wb') as f:
    np.save(f, vectors_tracks)

Et nous pouvons la recharger avec le code suivant :

In [100]:
vectors_tracks=np.load("latent_positions.npy")

## Utilisation en complétion et évaluation
Nous pouvons maintenant nous servir de cet espace pour faire des suggestions. <span style="color:red">Construisez une fonction predict_batch qui prend en entrée un vecteur de numéro de morceaux (seeds), (s) un nombre de proposition a faire, les vecteurs des morceaux dans l'espace latent X et un kd-tree permettant d'accélérer les calculs de plus proche voisins. Pour faire ses propisitions cette fonctions retournera les indices des s plus proche voisins de chaque seeds. </span> Pour que ces predictions ne prennent pas trop de temps vous vous servirez d'un kd-tree (disponnible dans scikit learn) pour accélrer la recherche des plus proche voisins.

In [101]:
from sklearn.neighbors import KDTree
kdt = KDTree(vectors_tracks, leaf_size=10, metric='euclidean')

In [102]:
def predict_batch(seeds,k,X,kdt):
    #return [kdt.query(X[i:i+1], k = 10) for i in seeds]
    preds = []
    for i in seeds : 
        x = kdt.query(X[i:i+1], k = 10)[1]
        preds.append(x)
        print(x)
    return preds

<span style="color:red">Utilisez cette fonction pour proposer des morceaux pour compléter les playlist du jeu de données de validation (les seeds correspondent à la première colone de play_val).</span>

In [103]:
indexes = predict_batch(play_val[:,0],10,vectors_tracks,kdt)

[[ 79029  57808  67834 117474   7249 120119  27292 100453  21749  65116]]
[[   532  16575  95539 120999  32801  13920  53593  38418   7669  90911]]
[[50537 53902 34042 97577 68684 84076 34792 59611 69580 85550]]
[[ 79202  55713  56107 121037   9831  30874  41438  84203 122630  96498]]
[[ 68426  16660  46649 122351  36729  86615 119597 121234  22055  82342]]
[[   569 114669 116072  83911  92219  10712  66936   8407  21758  87389]]
[[ 45641 104978 108474  40805 113272  67164  74093  41095  40051  36974]]
[[ 59046  56007  64079  24680 103437 112607  32312  77848  27838  86539]]
[[  5675 118668  95852      3  67679  50309  78556  66147  42086  78135]]
[[ 48042  24750  44116  79464 106838  55586  40330 104393 114548  57862]]
[[  152 22439  4005 44346 23080 27176 65053 52741 22409 67198]]
[[ 11736  94066 108659  47248  97489  40828  73720  16284  27118  92881]]
[[    30  30800    651  77466 113265  13703  41809  84920  49396  76110]]
[[  9851  73445  41188  13005  78912  29008  78544  24785 

[[ 42914 102000 101248  31150  95464  70600  39691  41454  46274  98717]]
[[  7090  72284 102218  58163 112146  36069  39796  65900  18833  57966]]
[[  100 47470 52909 28302 72256 89640 55612 14581 26647 98159]]
[[ 11037  93083 113872  63913 114291  26576  58493  26320  30689  81167]]
[[    67  80951  65686  30268  60100  94908  61540 117135  51008  56216]]
[[ 19447   6690  22527 108739  70250 102257 114718  23013  65716  85976]]
[[112256  68694  54382  42635  80413  98316  77351  40040   8664 114409]]
[[ 38705  62712  88709  71963  65983  26552  93260   6431 107460  81911]]
[[  4626  51012  92774  97731 118768  32868  24487  84068 118689 122830]]
[[ 20544  40464  60676  92650  21766 112298 109524  55644  22066  65919]]
[[11621 14382  3966 21386 76400 98347   960 51935 26057 77015]]
[[  7422  92963  97279  82703  31207  95801  54224 109639  36610  22171]]
[[   387  36368  47774  29864 103734  21873  17097  27417 112773  97890]]
[[114819  33226  75872 106124 101154   5062 106284  93654 

[[ 54192  89242 117941  27428 108884  85208 110592 101149  65840  62672]]
[[    89  22954 116744  74062 102878  28378  57155  37721  36642 111892]]
[[ 69767 109369  25359  37199  99931   3159 123230 100529  76214  54671]]
[[ 15794  10000  54741  41364  75272  22637 121220  67162 110822  49700]]
[[  1397  49456  33232 119782  91519  99912 107546  56410   6888   6431]]
[[   740  70178  62867 117927  70471 102045  61151  83235 113635  14444]]
[[ 24548  81498  97262  23639 111628  40824  26911  27842 104269  82084]]
[[ 32442 118579  41677  11380  12804 122300  85244 114823  27054  68651]]
[[    28  39135  62538  93986  77448  93143  45899 108122  69561  92440]]
[[ 23589  69490 110958   2439 119072  44307 104627  85034  45382  61752]]
[[ 2101 60421 69339  5159 28923 52884 35539 61249  2998 63072]]
[[  5708  82883  96087  89374  39471  40310 101477 116478  75734  24629]]
[[62730 58586 89116 54759 97127 37668 36433 39515 45110   864]]
[[  3947 115360  56592  22819 101970  15216  33902  30472 

[[ 11162  56862  18428 104709  28104  52811  95961   1910  42086  83525]]
[[ 67615  95521  68476  61380  55800  22174 100422  21358  94423  33490]]
[[ 63800  44707  15687  71864 104115  89956  86214  30206 117272 101820]]
[[23069 22677 97499 54712 16059 68068 18593 29400 15970 47236]]
[[  5060 115525  45683 111736  21967  16012  60858  74140 101563  24270]]
[[   126  61573  46939 112260  45283   2135  70883  51123  12492  73480]]
[[  1448  84353  25713  87898  17641   9037   1470  47568 109738  60488]]
[[84121 81450 34786 56761 66504 63190 18976 63428 35226 15583]]
[[ 54118 110341  16610 117379  27978  89312 117520   1939  57987  49170]]
[[   360 103827  92855  77914  97290  16451  77935 116470  82990  26674]]
[[17069 87515 60932 54477 48899  9044 40164 79846 91059 45237]]
[[ 88995  50913  11299  76147 114933  51793  25679  37445 121882   2601]]
[[  1122 115350  95744  39707  37670 114399  99895  85632  79772  31929]]
[[  3182  90350  68705  91366  91073 111249  76818 119200 117159  94

[[   904  41849  83655 116606  52654  82637  91656  79541  33393  32831]]
[[67795 95865 75628 21721 48231 60174 79065 88134 93729 17198]]
[[83132 48496 35351 63824 75125 80764 61474 89429 10501 28986]]
[[ 30207  72434  24115  46911  12944 107976  15407  45823  21776 113180]]
[[  7764  74531   7679  27703 114595   1647 109370  92563  53760  72314]]
[[     0   7269  28132   8325  29268  92833  48076  42638 100530  28702]]
[[ 36525  71468  71601  53561  64442 113475   2687   9287  91674  52710]]
[[28601  5334 97351 11570 36580 43147 42314 30083 59614  2010]]
[[ 22261  20417  38829  62150 102642  77755  13789  93618 116854  18279]]
[[  2511  87541  32233  82294  96913  42101  43301  49267 106826 106908]]
[[  7088  66987 104785  58883  31497  38058  10632  88508  10313   5411]]
[[  2479  16166  39436  73001 113179   5369  69597  91513  47786  20665]]
[[ 2083 32113  5266 25683 65226 97145 54843 81192 92066 30938]]
[[    92  72013  69586  44831 109481  82436  47291  57538   7676  36130]]
[[10

...

[[  3620  66841 104124 112382  97756 120548 112802 111693  87918  99344]]
[[61219 49355 45171 31446 23322 56481 50338 90246 22435 11505]]
[[  4863  10532  90918  84614  99932 101183 116845  28742  20178  51606]]
[[ 40692  70012  91288  15531  85935  65377   1545 110828  55237  64958]]
[[ 1816 91319 79096 98441 27905  6108 37944 93510 35813  1402]]
[[   982  74663  30603  89365  87673  35556  71013  20964  91529 117506]]
[[ 88119  84838  56155  81364  58167  85026  89199 105988  84242   4876]]
[[  2615  86890 111008  19000  67177  83003  96431 107918  37560  43888]]
[[  1767  33229  47599  11661  26186  17078  36490  77576 102963  79414]]
[[   901 114160 121300  93908  57005  70410   8827  73267 115646  27897]]
[[ 11434  99200  27180 102079   5895  29990 100844  15838  11309  39870]]
[[ 11458 106563 109825  29917  66295  87119  92242   3603  35425  11790]]
[[ 14908 103481 103501  53619 114312   5376 123023  26017  42813  34428]]
[[  7868  90014  70037   3174  44489   6058  16867 120990 

[[ 19284  92729  75335  36533  54584  66524 110896  23792  24030  44155]]
[[ 10917   4176  35727  73142 121802 111804  93593 109826  33323  43414]]
[[ 9602 49572 80321 15941 14008 52672 86399 83089 32151 57377]]
[[15825   269 28177 40923 75731  2274 12199 88752 18538 56506]]
[[112153  73149  89752 113394  92746 102154  50742  19558  48381   8035]]
[[  3685  77502  73862  65670  32490 112714  42918  80962   6014  48610]]
[[  4637  30151  55163  82787 105130 112698 102214  49783  31764  80328]]
[[     5  30244  26368 122595  89958  82422 104765  99383  64361 119191]]
[[    49  81007  36028 114006  84624   9899  98584  39475  54196  43714]]
[[  5161 117266   8157  94200 114206  51584 106456  86133  25044 100870]]
[[ 15695 116152 109802  85381  98374  17442  74051  97092  23444  47138]]
[[ 13866  27093  97890 119354   3968   6615  80570  65891  12026  34976]]
[[ 78768  88249 108777   6935  69541  83086   6309   5538  74921  19095]]
[[   173 103647  18825  68423  11121  66413  17445  15768 

[[ 71091  10164  73511  80139  34124  68873 122590  79608 111146 100739]]
[[  772 72635 69739 72926 46071 60480 14172 10087 58415 95172]]
[[   596 116959  18778  17468  40422 113642  47867    106 109874  27198]]
[[  3752  37387   5370  79416  71341  92127  41184  82678  97418 115303]]
[[  2194  82430  65961   8424  11711  57465  54507  11089 119704  31729]]
[[  1605  30742  43668  92532 111865  15657   3585  73955 111173  82093]]
[[ 15731  64983  19827  21679 103296  80102  97884  50637 105633 102754]]
[[82141 40894 22411 13418  5926 81444 73049 57986 23274 82462]]
[[   762  40039 107367  34567  63734  76380 119372  34069  35872  96116]]
[[ 75226  61939  40529  29951  98072 118223 101531  47313  13256 116121]]
[[ 30393 111629  28334  14069  28383  51810  34952  77521  95982  97522]]
[[ 23015  36806 115247  76398   6907  71155  68045  58134  80267  35332]]
[[  3689  37835 117253 103374  38269  90326  63073  19249 114328    239]]
[[123088  34581  58124  45436  51931   5858  85425  67943 

[[  246 85810 90936 79602 80695  9787 64613 70406 43224 30673]]
[[   757  83410  55154  56952  20118 115306   4349  73342  49347  60206]]
[[  1001 119462  58370  12552 101073  26243  54415  33149 106881  37995]]
[[   324  45646  86513 117747  86990  13774  63133  88185  56029 102746]]
[[ 41765  47812  18318 122738    837 102570  10664   3772  21952  26655]]
[[35748 45525 90474 86010 15876 10296 68046  3395 15049 16907]]
[[ 96340  28529  23716  44281   7137  35587 100753  76466  66023  33370]]
[[ 17071   4041  23353  77963  53461  22310  64836 105469 115449 103442]]
[[ 56564  94237  90352 110541  14460 115450  79630  74552  60188  42837]]
[[66736 19680 18309 51500  6244 99316 13325 99312 69956 88745]]
[[ 87879  33519  11073 108210  89581  93322  40170  78723  36586 109785]]
[[ 23904   2493  87875 120096    473  58851  99458  74365  10814 112900]]
[[ 30297 109661  45263  92831  89277  19652  88543  92859  91729  25151]]
[[113667  66509  36599  76472  19039 113102    116 107567 100439  40

[[    49  81007  36028 114006  84624   9899  98584  39475  54196  43714]]
[[   314   8038  70570 101441  78203  64531  41060  77225  69586  22986]]
[[ 27511  88944  96172  98763  62276  64055  99807  76803  67731 121438]]


<span style="color:red">Comparez ces suggestions avec la seconde colonne de play_val (les morceaux effectivement présents). Pour cela vous calculerez le hit@10 qui vaut 1 si le morceau effectivement présent dans la playlist fait partie des 10 propositions (ce score étant moyenné sur l'ensemble de validation) et le NDCG@10 (Normalized Discounted Cumulative Gain) qui prend en compte l'ordre des propositions. Ce second score vaut $1/log2(k+1)$ si la proposition k (k entre 1 et 10) est la proposition correcte et 0 si aucne proposition n'est correcte. Comme précedement vous calulerez le score moyen sur l'ensemble de validation. </span>


In [104]:
from math import * 

def calculate_ndgc_at_10(indexes, real_songs) :
    score = 0
    for i in range(len(indexes)) :
        for j in range(len(indexes[i][0])) :
            if  real_songs[i] == indexes[i][0][j] :
                score += 1 / log(((j + 1) + 1), 2)
    return score / len(indexes)

NDGCatK = calculate_ndgc_at_10(indexes, play_val[:,0])
print(NDGCatK)

1.0


In [105]:
def calculate_hit_at_10(indexes, real_songs) :
    score = 0
    for i in range(len(indexes)) :
        if  real_songs[i] in indexes[i][0] :
            score += 1
    return score / len(indexes)

HitatK = calculate_hit_at_10(indexes, play_val[:,0])
print(HitatK)

1.0


## Tunning des hyper paramètres

<span style="color:red">Vous pouvez maintenant essayer de faire varier les hyper paramètres pour améliorer vos performances. Attention au temps de calcul préparez une grille avec une dizaine de configurations différentes et évaluez chacune d'entre elles sur votre ensemblede validation.
Evaluez les performances finales de la meilleure configuration trouvée sur l'ensemble de test. N'oubliez pas de sauvergader vos résultats.</span>



In [10]:
# dimension de l'espace latent
vector_dim = 30
# taille de la fenêtre de voisinage
window_width = 5 # ---- Modified 
# sur-échantillonage des exemples négatifs
neg_sample = 5
# taille des mini-batch
min_batch_size = 50
# coeff pour la loi de tirage des exemple negatif
samp_coef = 0.5
# coeff pour le subsampling
sub_samp = 0.00001

# loss: 0.4487 - accuracy: 0.8344

In [17]:
# dimension de l'espace latent
vector_dim = 10 # ---- Modified 
# taille de la fenêtre de voisinage
window_width = 3
# sur-échantillonage des exemples négatifs
neg_sample = 5
# taille des mini-batch
min_batch_size = 50
# coeff pour la loi de tirage des exemple negatif
samp_coef = 0.5
# coeff pour le subsampling
sub_samp = 0.00001

# loss: 0.4502 - accuracy: 0.8333

In [25]:
# dimension de l'espace latent
vector_dim = 30
# taille de la fenêtre de voisinage
window_width = 3
# sur-échantillonage des exemples négatifs
neg_sample = 3 # ---- Modified 
# taille des mini-batch
min_batch_size = 50
# coeff pour la loi de tirage des exemple negatif
samp_coef = 0.5
# coeff pour le subsampling
sub_samp = 0.00001

# loss: 0.5654 - accuracy: 0.7473

In [32]:
# dimension de l'espace latent
vector_dim = 30
# taille de la fenêtre de voisinage
window_width = 3
# sur-échantillonage des exemples négatifs
neg_sample = 5
# taille des mini-batch
min_batch_size = 10 # ---- Modified 
# coeff pour la loi de tirage des exemple negatif
samp_coef = 0.5
# coeff pour le subsampling
sub_samp = 0.00001

# loss: 0.4450 - accuracy: 0.8350

In [39]:
# dimension de l'espace latent
vector_dim = 40 # ---- Modified 
# taille de la fenêtre de voisinage
window_width = 2 # ---- Modified 
# sur-échantillonage des exemples négatifs
neg_sample = 3 # ---- Modified 
# taille des mini-batch
min_batch_size = 10 # ---- Modified  
# coeff pour la loi de tirage des exemple negatif
samp_coef = 0.3 # ---- Modified 
# coeff pour le subsampling
sub_samp = 0.0001 # ---- Modified 

# loss: 0.3035 - accuracy: 0.9460

In [46]:
# dimension de l'espace latent
vector_dim = 40 # ---- Modified 
# taille de la fenêtre de voisinage
window_width = 3
# sur-échantillonage des exemples négatifs
neg_sample = 5
# taille des mini-batch
min_batch_size = 50
# coeff pour la loi de tirage des exemple negatif
samp_coef = 0.5
# coeff pour le subsampling
sub_samp = 0.00001

# loss: 0.4403 - accuracy: 0.8396

In [53]:
# dimension de l'espace latent
vector_dim = 30
# taille de la fenêtre de voisinage
window_width = 2 # ---- Modified 
# sur-échantillonage des exemples négatifs
neg_sample = 5
# taille des mini-batch
min_batch_size = 50
# coeff pour la loi de tirage des exemple negatif
samp_coef = 0.5
# coeff pour le subsampling
sub_samp = 0.00001

# loss: 0.4545 - accuracy: 0.8296

In [61]:
# dimension de l'espace latent
vector_dim = 30
# taille de la fenêtre de voisinage
window_width = 3
# sur-échantillonage des exemples négatifs
neg_sample = 5
# taille des mini-batch
min_batch_size = 75 # ---- Modified 
# coeff pour la loi de tirage des exemple negatif
samp_coef = 0.5
# coeff pour le subsampling
sub_samp = 0.00001

# loss: 0.4322 - accuracy: 0.8483

In [68]:
# dimension de l'espace latent
vector_dim = 30
# taille de la fenêtre de voisinage
window_width = 3
# sur-échantillonage des exemples négatifs
neg_sample = 5
# taille des mini-batch
min_batch_size = 50
# coeff pour la loi de tirage des exemple negatif
samp_coef = 0.7 # ---- Modified 
# coeff pour le subsampling
sub_samp = 0.00001

# loss: 0.4577 - accuracy: 0.8290

In [76]:
# dimension de l'espace latent
vector_dim = 30
# taille de la fenêtre de voisinage
window_width = 3
# sur-échantillonage des exemples négatifs
neg_sample = 5
# taille des mini-batch
min_batch_size = 50
# coeff pour la loi de tirage des exemple negatif
samp_coef = 0.5
# coeff pour le subsampling
sub_samp = 0.000001 # ---- Modified 

# loss: 0.4580 - accuracy: 0.8287

## Bonus, un peu de musique

Le fichier TrackArtists contient des méta.données sur les morceaux et les artiste pour un sous ensemble des 300000 morceaux présent dans le dataset. Nous pouvons nous en servir pour recherchez le numéro d'un morceau a partir de son titre:

In [ ]:
import pandas as pd
tr_meta=pd.read_csv("./TracksArtists.csv")
joindf = pd.DataFrame({"track_id":tracks_list_ordered[:Vt],"index":range(Vt)})
meta = tr_meta.merge(joindf, left_on="track_id",right_on="track_id")
meta.set_index("index",inplace=True)
meta[["title","name","preview","track_id"]]

In [ ]:
def find_track(title):
    return meta.loc[meta["title"]==title,:].index[0]

tr=find_track("Hexagone")
tr

## Radio

L'api de deeezer permet de récupérer des informations sur les morceaux du dataset a partit de leur id deezer. Parmis ces infos lorsqu'elle est disponnible une url d'écoute d'un extrait gratuit est fournies.

In [ ]:
import urllib.request, json 
def gettrackinfo(number):
    track_url =  "https://api.deezer.com/track/{}".format(tracks_list_ordered[number].split("_")[1])
    with urllib.request.urlopen(track_url) as url:
        data = json.loads(url.read().decode())
    return data
track_apidata = gettrackinfo(find_track("Hexagone"))
track_apidata

Nous pouvons donc nous en servir pour écouter un extrait :

In [ ]:
from IPython.display import display, Audio, clear_output
display(Audio(track_apidata["preview"],autoplay=True))

<span style="color:red">Créez une fonction radio qui prend en entrée un numéro de morceau dans le dataset et lance une serie de nb_track morceaux en tirant aléatoirement dans le voisinage du morceau courant le morceau suivant a écouter. La taille du voisinage sera paramétrable et vous supprimerez des propositions les morceaux déjà écouté. Vous traiterez les exceptions si le morceau ne dispose pas d'extrait disponnible. Vous povez supprimer le morceau courant avec la fonction clear_display.</span>

In [ ]:
import time
def start_radio(seed,nb_candidates,duration,nbsteps=20):
    print(meta.loc[seed,"title"])
    display(Audio(meta.loc[seed,"preview"],autoplay=True))
    time.sleep(duration)
    clear_output()
    already_played = [seed]
    for i in range(nbsteps):
        try:
            # TODO
        except:
            print("track not found")
            pass
        clear_output()

In [ ]:
start_radio(find_track("Hexagone"),5,5,10)